In [7]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
from matplotlib import animation as anm
from IPython.display import display, HTML
%matplotlib inline

In [8]:
env = gym.make('CartPole-v0')
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [9]:
def  env_render(env_vis):
    plt.figure()
    plot  =  plt.imshow(env_vis[0])
    plt.axis('off')

    def  animate(i):
        plot.set_data(env_vis[i])
        anim  =  anm.FuncAnimation(plt.gcf(), animate, frames=len(env_vis), interval=20, repeat=True, repeat_delay=20)
        display(display_animation(anim,  default_mode='loop'))

In [10]:
gamma = 0.99

def discount_rewards(r):
    """ Take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(r.size)):
        running_sum = running_sum * gamma + r[i]
        discounted_r[i] = running_sum
    return discounted_r

In [11]:
class agent():
    def __init__(self, learning_rate, state_size,
                 action_size, hidden_layer_size):
        
        # These four lines establish the feed-forward part of the network.
        # The agent takes a state and produces an action.
        self.state_input = tf.placeholder(shape=[None, state_size],
                                          dtype=tf.float32)
        
        hidden_layer = slim.fully_connected(inputs=self.state_input,
                                            num_outputs=hidden_layer_size,
                                            activation_fn=tf.nn.relu,
                                            biases_initializer=None)
        
        self.output = slim.fully_connected(inputs=hidden_layer,
                                           num_outputs=action_size,
                                           activation_fn=tf.nn.softmax,
                                           biases_initializer=None)
        
        self.chosen_action = tf.argmax(self.output, 1)
        
        # These next six lines establish the training proceedure.
        # We feed the reward and chosen action into the network
        # in order to compute the loss and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],
                                            dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],
                                            dtype=tf.int32)
        # Not 100% sure yet what these ops do.
        # Something to with reordering the output tensor which may be 2D
        # if multiple states are pushed through
        o_shape = tf.shape(self.output)
        self.indexes = tf.range(0, o_shape[0]) * o_shape[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]),
                                             self.indexes)
        
        # Policy loss function
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
        # Make a list of placeholder: each element is a placeholder
        # for a trainable variable in the network
        trainable_variables = tf.trainable_variables()
        self.gradient_holders = []
        for i, x in enumerate(trainable_variables):
            placeholder = tf.placeholder(dtype=tf.float32,
                                         name=str(i)+'_holder')
            self.gradient_holders.append(placeholder)
        
        # Create a bunch of gradients, one for each trainable variable
        self.gradients = tf.gradients(self.loss, trainable_variables)
        
        # Define an optimizer.
        # We define the model update function using the apply_gradients
        # function instead of using minimize as that allows us to
        # control the number of steps taken per update.
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,
                                                          trainable_variables))

In [12]:
tf.reset_default_graph()

myAgent = agent(learning_rate=1e-2,
                state_size=n_states,
                action_size=n_actions,
                hidden_layer_size=8)

total_episodes = 5000
max_steps_per_episode = 999
network_update_frequency = 5

init = tf.global_variables_initializer()

env_vis = []

with tf.Session() as sess:
    sess.run(init)
    
    total_reward = []
    total_length = []
    
    gradient_buffer = [x*0 for x in sess.run(tf.trainable_variables())]
    
    for episode in range(total_episodes):
        state = env.reset()
        running_reward = 0
        episode_history = []
        for j in range(max_steps_per_episode):
            if episode % 100 == 0 and episode != 0:
                env.render()
            # Probabilistically choose an action give the network outputs.
            action_distribution = sess.run(myAgent.output,
                                           feed_dict={myAgent.state_input:[state]})
            action = np.random.choice(action_distribution[0],
                                      p=action_distribution[0])
            action = np.argmax(action_distribution == action)
            
            # Perform the action in the environment and get new state
            # and reward
            new_state, reward, done, _ = env.step(action)
            episode_history.append([state, action, reward, new_state])
            state = new_state
            running_reward += reward
            if j == max_steps_per_episode-1:
                print('foo')
            
            if done == True:
                # Episode finished
                # Process the rewards into discounted form and then run the
                # feed forward network again to get the gradients produced
                # by the experiences and the reward. These gradients are
                # added to the gradient buffers, which will be used to update
                # the network in one batch.
                episode_history = np.array(episode_history)
                episode_history[:,2] = discount_rewards(episode_history[:,2])
                feed_dict = {myAgent.reward_holder:episode_history[:,2],
                             myAgent.action_holder:episode_history[:,1],
                             myAgent.state_input:np.vstack(episode_history[:,0])}
                gradients = sess.run(myAgent.gradients, feed_dict=feed_dict)
                gradient_buffer = [gradient_buffer[i] + x for i, x in enumerate(gradients)]
                
                # If we've run the prescribed number of episodes then
                # we can update the network weights with the buffered
                # gradients. Once we've updated the network we reset the
                # gradient buffer to 0.
                if episode % network_update_frequency == 0 and episode != 0:
                    feed_dict = dict(zip(myAgent.gradient_holders,
                                         gradient_buffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    gradient_buffer = [x*0 for x in gradient_buffer]
                
                total_reward.append(running_reward)
                total_length.append(j)
                break
                
            # Update our running tally of scores
        if episode % 100 == 0:
            print("Episode %i: %f" % (episode, np.mean(total_reward[-100:])))

C:\Users\Andre\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Episode 0: 23.000000
Episode 100: 24.840000
Episode 200: 32.450000
Episode 300: 36.510000
Episode 400: 61.610000
Episode 500: 107.540000
Episode 600: 136.880000
Episode 700: 162.940000
Episode 800: 179.490000
Episode 900: 180.460000
Episode 1000: 188.860000
Episode 1100: 189.090000
Episode 1200: 180.800000
Episode 1300: 174.900000
Episode 1400: 175.650000
Episode 1500: 165.100000
Episode 1600: 175.870000
Episode 1700: 172.590000
Episode 1800: 156.940000
Episode 1900: 174.510000
Episode 2000: 192.380000
Episode 2100: 198.030000
Episode 2200: 198.450000
Episode 2300: 199.990000
Episode 2400: 198.770000
Episode 2500: 195.670000
Episode 2600: 198.600000
Episode 2700: 199.230000
Episode 2800: 199.660000
Episode 2900: 199.560000
Episode 3000: 199.480000
Episode 3100: 199.320000
Episode 3200: 198.050000
Episode 3300: 199.020000
Episode 3400: 197.700000
Episode 3500: 194.990000
Episode 3600: 199.130000
Episode 3700: 199.060000
Episode 3800: 195.220000
Episode 3900: 195.180000
Episode 4000: 199